## Настройка окружения

In [1]:
# загружаем библиотеки
import pandas as pd
from sqlalchemy import create_engine 

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [3]:
# функция, выполняющая SQL-запрос
def ask(sql):
    return pd.read_sql(sql,con=engine)

## Цели исследования

1. Проанализировать базу данных для того чтобы сформулировать ценностное предложение для нового продукта
2. Изучить информацию о книгах, издательствах, авторах и пользовательских обзоров книг.

## Исследование таблиц

In [4]:
# присваиваем переменной таблицу с книгами
books_sql = '''
SELECT *
FROM books
ORDER BY
 book_id ASC
LIMIT 5
'''
ask(books_sql)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
# присваиваем переменной таблицу с авторами
authors_sql = '''
SELECT *
FROM authors
ORDER BY
 author_id ASC
LIMIT 5
'''
ask(authors_sql)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
# присваиваем переменной таблицу с издательствами
publishers_sql = '''
SELECT *
FROM publishers
ORDER BY
 publisher_id ASC
LIMIT 5
'''
ask(publishers_sql)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [7]:
# присваиваем переменной таблицу с рейтингами
ratings_sql = '''
SELECT *
FROM ratings
ORDER BY
 rating_id ASC
LIMIT 5
'''
ask(ratings_sql)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [8]:
# присваиваем переменной таблицу с обзорами
reviews_sql = '''
SELECT *
FROM reviews
ORDER BY
 review_id ASC
LIMIT 5
'''
ask(reviews_sql)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Вывод об исследовании таблиц

Подключение к базе данных успешно. Все таблицы сохранены в своих переменных. 

## Анализ данных

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.


In [9]:
# сколько книг вышло после 1 января 2000 года
books_cnt = '''
SELECT COUNT(DISTINCT book_id) AS books_count
FROM books
WHERE publication_date >= '2000-01-01'
'''
ask(books_cnt)

,books_count
0,821


После 1 января 2000 года вышла 821 книга

In [10]:
# количество обзоров и средняя оценка для каждой книги
rev_cnt = """
  
WITH books AS (
    SELECT book_id, 
    title 
    FROM books
),
g_ratings AS (
    SELECT 
    book_id,
    AVG(rating) AS avg_rating
    FROM ratings
    GROUP BY book_id
),
g_reviews AS (
    SELECT 
    book_id,
    COUNT(review_id) AS cnt_review
    FROM reviews
    GROUP BY book_id
)
SELECT 
books.title,
g_ratings.avg_rating,
g_reviews.cnt_review
FROM books
LEFT JOIN g_ratings ON books.book_id = g_ratings.book_id
LEFT JOIN g_reviews ON books.book_id = g_reviews.book_id
ORDER BY g_ratings.avg_rating DESC, g_reviews.cnt_review DESC


"""


ask(rev_cnt)

,title,avg_rating,cnt_review
0,The Cat in the Hat and Other Dr. Seuss Favorites,5.00,NaN
1,A Dirty Job (Grim Reaper #1),5.00,4.0
2,Moneyball: The Art of Winning an Unfair Game,5.00,3.0
3,School's Out—Forever (Maximum Ride #2),5.00,3.0
4,Women,5.00,2.0
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,2.25,3.0
996,Drowning Ruth,2.00,3.0
997,His Excellency: George Washington,2.00,2.0
998,Junky,2.00,2.0


* Вся тысяча книг
* В основном запросе к таблице books присоединяются результаты двух подзапросов с подсчетом количества обзоров и средней оценкой каждой книги.
* Есть книги без отзывов

In [11]:
# опредеяем издательство, которое выпустило наибольшее число книг толще 50 страниц
pub = '''
SELECT
 publishers.publisher_id AS id,
 publishers.publisher AS publisher,
 COUNT(DISTINCT books.book_id) AS number_of_books
FROM publishers
 LEFT JOIN books ON books.publisher_id = publishers.publisher_id
WHERE 
 books.num_pages > 50
GROUP BY
 id,
 publisher
ORDER BY
 number_of_books DESC
LIMIT 10
'''
ask(pub)

,id,publisher,number_of_books
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25
3,217,Penguin Classics,24
4,33,Ballantine Books,19
5,35,Bantam,19
6,45,Berkley,17
7,46,Berkley Books,14
8,284,St. Martin's Press,14
9,83,Delta,13


Больше всех книг толще 50 страниц выпустило издетельство "Penguin Books"

In [12]:
# определим автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками
aut = '''
SELECT 
  br.author,
  AVG(br.avg)
  FROM 
  (
    SELECT b.title,
           author,
           AVG(rating)
           FROM books AS b
           JOIN authors AS a ON b.author_id=a.author_id
           JOIN ratings AS r ON b.book_id=r.book_id
           GROUP BY b.book_id,
                    author
           HAVING COUNT(r.book_id) >= 50
           ORDER BY AVG(rating) 
    ) AS br
GROUP BY br.author
ORDER BY AVG(br.avg) DESC
'''

ask(aut)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Автор с самой высокой средней оценкой книг J.K. Rowling/Mary GrandPré имеет самое большое число оценок в топ-10 авторов

In [13]:
# считаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок
rev = '''
SELECT
 AVG (number_of_reviews) AS AVG_count
 FROM
 (
   SELECT
    COUNT (review_id) AS number_of_reviews
   FROM
    reviews
   INNER JOIN
   (SELECT
     username,
     COUNT (rating_id) AS number_of_ratings
    FROM
     ratings
    GROUP BY
     username
    HAVING COUNT (rating_id) > 50 ) AS ratings_users ON ratings_users.username = reviews.username
    GROUP BY
    reviews.username
 ) AS number_of_reviews
'''
ask(rev)

,avg_count
0,24.333333


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24

## Ответы на задания

1. После 1 января 2000 года вышла 821 книга
2. В основном запросе к таблице books присоединяются результаты двух подзапросов с подсчетом количества обзоров и средней оценкой каждой книги.
3. Больше всех книг толще 50 страниц выпустило издетельство "Penguin Books"
4. Автор с самой высокой средней оценкой книг J.K. Rowling/Mary GrandPré имеет самое большое число оценок в топ-10 авторов
5. Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24